# Assignment 3 - NLP 
# Name: Muhammad Ahsan Farooqui
# Roll Number: 20I-2207
---

# Importing Libraries and Data

In [1]:
import numpy as np
import pickle
import spacy
import random
import math
import re


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
import csv

with open('./Data/iqbal.txt', 'rt', encoding="utf-8") as f:
    reader = csv.reader(f)
    passage = list(reader)
f.close()

with open('./Data/ghalib.txt', 'rt', encoding="utf-8") as f:
    reader = csv.reader(f)
    passage.append(list(reader))
f.close()

with open('./Data/faiz.txt', 'rt', encoding="utf-8") as f:
    reader = csv.reader(f)
    passage.append(list(reader))
f.close()


passage = [item for sublist in passage for item in sublist]


## Removing English, punctuation marks and other special characters

In [4]:
for i in passage:
  b = re.findall("[a-zA-Z]",str(i))
  if(len(b)>2):
    print(i)

['Musafir K Rastay Badaltay Rahay']
['Muqaddar Main Chalna Tha Chaltay Rahay']
['Muhabbat Adawat Wafa Berukhi']
['Kiray K Ghar Thay Badalty Rahay']


In [5]:
english_verses = []
for i in passage:
  b = re.findall("[a-zA-Z]",str(i))
  if(len(b)>2):
    english_verses.append(i)

passage = [x for x in passage if x not in english_verses]

In [6]:
replace_items = ["['","']","[","]","'","“","‘",'"',"’","'",'؛','،','٫','؟','۔','٪','!',"%"," ِ",".",":","(",")","*"]
for i in range(len(passage)):
  if(len(passage[i])>0):
    a = passage[i]
    for j in replace_items:
      a = str(a).replace(j,"")
      a = " ".join(a.split())
    passage[i]= a
    


---



## Creating Model for Spacy tokenization for Urdu

In [7]:
nlp = spacy.blank('ur')

# Defining Models

## Defining Unigram Model

In [8]:
def create_unigrams(passage):
  unigrams = [nlp("شششش "+(str(p).replace("[","").replace("]","").strip())+" خخخخ") for p in passage if(len(p)>0)]
  unigrams = [item for sublist in unigrams for item in sublist]
  unigrams = np.array(unigrams)
  unigrams = [str(x).strip() for x in unigrams]
  unique_elements, counts_elements = np.unique(unigrams, return_counts=True)
  unigram_dict_probability = {}
  unigram_dict_counts = {}

  for i,j in zip(unique_elements,counts_elements):
    unigram_dict_probability[i] = j/sum(counts_elements)
  unigram_dict_probability = dict(sorted(unigram_dict_probability.items(), key=lambda item: item[1],reverse=True))

  for i,j in zip(unique_elements,counts_elements):
    unigram_dict_counts[i] = j
  unigram_dict_counts = dict(sorted(unigram_dict_counts.items(), key=lambda item: item[1],reverse=True))

  #Resetting the tokens
  unigram_dict_probability["<s>"] = unigram_dict_probability["شششش"]
  del unigram_dict_probability["شششش"]

  unigram_dict_probability["</s>"] = unigram_dict_probability["خخخخ"]
  del unigram_dict_probability["خخخخ"]


  #Resetting the tokens
  unigram_dict_counts["<s>"] = unigram_dict_counts["شششش"]
  del unigram_dict_counts["شششش"]

  unigram_dict_counts["</s>"] = unigram_dict_counts["خخخخ"]
  del unigram_dict_counts["خخخخ"]
  return unigram_dict_counts,unigram_dict_probability,unigrams

In [9]:
def get_unigram_count(word,unigram_dict_counts):
  try:
    return unigram_dict_counts[word]
  except:
    return 0

In [10]:
def get_unigram_word(prev_word=0,sentence_length=5,unigram_dict_probability=0,unigram_sentence = []):
  length = len(unigram_dict_probability.keys())
  if sentence_length==0:
    print("")
    pass
  else:
    if(prev_word==0):
      word = length*random.random()
      selectedword = list(unigram_dict_probability.keys())[int(word)]
      unigram_sentence.append(selectedword)
      print(selectedword,end=" ")
    else:
      prev_word_position = list(unigram_dict_probability.keys()).index(prev_word)
      selectedword = list(unigram_dict_probability.keys())[int(prev_word_position-1)]
      unigram_sentence.append(selectedword)
      print(selectedword,end=" ")
    get_unigram_word(sentence_length = sentence_length-1,unigram_dict_probability=unigram_dict_probability,unigram_sentence=unigram_sentence)
  return unigram_sentence

In [11]:
def generate_unigram_sentence(sentence_length,unigram_dict_probability,unigram_sentence):
  sentence = get_unigram_word(prev_word=0,sentence_length=sentence_length,unigram_dict_probability=unigram_dict_probability,unigram_sentence=unigram_sentence)
  return sentence

### Evaluation of Unigrams

In [12]:
def calculate_perplexity_unigram(sentence,unigram_probability):
  probability = 1
  probability_log = 0
  for i in sentence:
    probability = probability*unigram_dict_probability[i]
    probability_log = probability_log + math.log(unigram_dict_probability[i])
  return probability,-1*probability_log,(1/(-1*probability_log))**len(sentence)
    


## Defining Bigram Model

In [13]:
def create_bigrams(passage, unigram_dict_counts,backward=False):
  bigrams = []
  dictionary = []
  urdu_characters = ['آ','أ','ا','ب','پ','ت','ٹ','ث','ج','چ','ح','خ','د','ڈ','ذ','ر','ڑ','ز','ژ','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ک','گ','ل','م','ن','ں','و','ؤ','ہ','ۂ','ۃ','ھ','ء','ی','ئ','ے','ۓ']
  urdu_punctuations = ['؛','،','٫','؟','۔','٪','!']
  for i in passage:
    data = nlp(str(i).replace("[","").replace("]","").strip())
    if(backward):
      data = list(data)[::-1]
    #print(data)
    for word in range(len(data)):
      if(word==0):
        a = "<s>"
        #print("len of a is ",len(a))
        b = str(data[word]).strip()
      elif(word==len(data)-1):
        b = "</s>"  
        a = str(data[word]).strip()
      else:
        a = str(data[word]).strip()
        b = str(data[word+1]).strip()
      if(((str(a)=="<s>") and (str(b)=="</s>"))| (len(a)==0) | (len(b)==0)|
        (((len(a)==1) and (str(a) not in urdu_characters)))|# and (str(a) not in urdu_punctuations))|
        (((len(b)==1) and (str(b) not in urdu_characters)))):# and (str(b) not in urdu_punctuations))):
        print(i)
        print(a)
        print(b)
        print("==")
        continue
      else:
        bigrams.append((a,b))
  bigrams_dict = {}
  bigrams_dict_probab = {}
  for i in bigrams:
    if(i in bigrams_dict.keys()):
      bigrams_dict[i] = bigrams_dict[i]+1
    else:
      bigrams_dict[i] = 1

  for i in bigrams_dict.keys():
    try:
      bigrams_dict_probab[i] = bigrams_dict[i]/get_unigram_count(i[0],unigram_dict_counts)
    except:
      print("Following word has 0 probability in unigrams", i[0])

  bigrams_dict = dict(sorted(bigrams_dict.items(), key=lambda item: item[1],reverse=True))
  bigrams_dict_probab = dict(sorted(bigrams_dict_probab.items(), key=lambda item: item[1],reverse=True))
  return bigrams,bigrams_dict,bigrams_dict_probab

In [14]:
def get_bigram_word(prev_word="123",sentence_length=0,total_length=10,bigrams_dict_probab=[],bigram_word=[]):
  
  if ((sentence_length==total_length) | (prev_word=="</s>")):
    print("")
  else:
    start_words_keys = [x for x in bigrams_dict_probab.keys() if x[0]=="<s>"]
    if(prev_word=="123"):
      length = len(start_words_keys)
      keys = length*random.random()
      selectedword = start_words_keys[int(keys)]

      print(selectedword[1],end=" ")
      bigram_word.append(selectedword[1])
      get_bigram_word(str(selectedword[1]),sentence_length = sentence_length+1,total_length=total_length,bigrams_dict_probab=bigrams_dict_probab,bigram_word=bigram_word)
    else:
      if(sentence_length > (total_length-3)):
        #keys = list(bigrams_dict_probab.keys())
        end_words_keys = [x for x in bigrams_dict_probab.keys() if x[1]=="</s>"]
        keys = end_words_keys
      else:
        end_words_keys = [x for x in bigrams_dict_probab.keys() if x[1]=="</s>"]
        keys = list(bigrams_dict_probab.keys())
        keys = list(set(keys)-set(end_words_keys)-set(start_words_keys))
      sets = [y for y in keys if y[0]==prev_word]
      nextword = [x for x in bigrams_dict_probab.keys() if x in sets]
      nextword = np.array(nextword)
      #print(nextword)
      nextword = nextword[:5]
      if(len(nextword)>0):
        vaal = len(nextword)*random.random()
        if(str(nextword[int(vaal)][1])!="</s>"):
          print(str(nextword[int(vaal)][1]),end=" ")
          bigram_word.append(str(nextword[int(vaal)][1]))
        get_bigram_word(str(nextword[int(vaal)][1]),sentence_length = sentence_length+1,total_length=total_length,bigrams_dict_probab=bigrams_dict_probab,bigram_word=bigram_word)
      else:
        get_bigram_word("123",sentence_length = sentence_length,total_length=total_length,bigrams_dict_probab=bigrams_dict_probab,bigram_word=bigram_word)    
  return bigram_word

In [15]:
def get_bigram_count(a,b,bigrams_dict):
  key = (a,b)
  return bigrams_dict[key]


## Defining Trigrams Model

In [16]:
def create_trigrams(passage,bigram_count):
  trigrams = []
  dictionary = []
  trigrams_dict = {}
  trigrams_dict_probab = {}
  urdu_characters = ['آ','أ','ا','ب','پ','ت','ٹ','ث','ج','چ','ح','خ','د','ڈ','ذ','ر','ڑ','ز','ژ','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ک','گ','ل','م','ن','ں','و','ؤ','ہ','ۂ','ۃ','ھ','ء','ی','ئ','ے','ۓ']
  for i in passage:
    data = nlp(str(i).replace("[","").replace("]","").strip())
    for word in range(len(data)-1):
      if(word==0):
        a = "<s>"
        b = str(data[word]).strip()
        c = str(data[word+1]).strip()
      elif(word==len(data)-2):
        c = "</s>"  
        a = str(data[word]).strip()
        b = str(data[word+1]).strip()
      else:
        a = str(data[word]).strip()
        b = str(data[word+1]).strip()
        c = str(data[word+2]).strip()
      if(((str(a)=="<s>") and (str(c)=="</s>"))| (len(a)==0) | (len(b)==0)| (len(c)==0) |
        (((len(a)==1) and (str(a) not in urdu_characters)))|# and (str(a) not in urdu_punctuations))|
        (((len(b)==1) and (str(b) not in urdu_characters)))|# and (str(b) not in urdu_punctuations))|
        (((len(c)==1) and (str(c) not in urdu_characters)))):# and (str(c) not in urdu_punctuations))):
        print(i)
        print("a=",a)
        print("b=",b)
        print("c=",c)
        print("==")
        continue
      else:
        trigrams.append((a,b,c))

  for i in trigrams:
    if(i in trigrams_dict.keys()):
      trigrams_dict[i] = trigrams_dict[i]+1
    else:
      trigrams_dict[i] = 1

  for i in trigrams_dict.keys():
    try:
      trigrams_dict_probab[i] = trigrams_dict[i]/get_bigram_count(i[0],i[1],bigram_count)
    except:
      print(i[0])

  trigrams_dict = dict(sorted(trigrams_dict.items(), key=lambda item: item[1],reverse=True))
  trigrams_dict_probab = dict(sorted(trigrams_dict_probab.items(), key=lambda item: item[1],reverse=True))
  return trigrams, trigrams_dict, trigrams_dict_probab

In [17]:
def get_trigram_word(first_word="123",second_word="123",sentence_length=0,total_length=6,trigrams_dict_probab=""):
  trigram_sentence = []
  global a
  if ((sentence_length==total_length) | (second_word=="</s>")):
    print("")
  else:
    start_words_keys = [x for x in trigrams_dict_probab.keys() if x[0]=="<s>"]
    if((first_word=="123") and (second_word=="123") ):
      length = len(start_words_keys)
      keys = length*random.random()
      selectedword = start_words_keys[int(keys)]

      print(selectedword[1],selectedword[2],end=" ")
      trigram_sentence.append(selectedword[1])
      trigram_sentence.append(selectedword[2])
      get_trigram_word(str(selectedword[1]),str(selectedword[2]),sentence_length = sentence_length+1,total_length=total_length,trigrams_dict_probab=trigrams_dict_probab)
    else:
      if(sentence_length > (total_length-3)):
        keys = list(trigrams_dict_probab.keys())
      else:
        end_words_keys = [x for x in trigrams_dict_probab.keys() if x[2]=="</s>"]
        keys = list(trigrams_dict_probab.keys())
        keys = list(set(keys)-set(end_words_keys)-set(start_words_keys))
      sets = [y for y in keys if (y[0]==first_word) and (y[1]==second_word)]
      nextword = [x for x in trigrams_dict_probab.keys() if x in sets]
      nextword = np.array(nextword)
      if(len(nextword)>0):
        vaal = len(nextword)*random.random()
        if(str(nextword[int(vaal)][2])!="</s>"):
          print(str(nextword[int(vaal)][2]),end=" ")
          trigram_sentence.append(str(nextword[int(vaal)][2]))
        get_trigram_word(str(nextword[int(vaal)][1]),str(nextword[int(vaal)][2]),sentence_length = sentence_length+1,total_length=total_length,trigrams_dict_probab=trigrams_dict_probab)
      else:
        get_trigram_word("123","123",sentence_length = sentence_length,total_length=total_length,trigrams_dict_probab=trigrams_dict_probab)    


# Running everything

## Creating Unigrams, Bigrams, Backward Bigrams and Trigrams

*Important: It may take some time to generate all models.*

In [18]:
unigram_dict_counts,unigram_dict_probability,unigrams = create_unigrams(passage)
bigrams,bigrams_dict,bigrams_dict_probab = create_bigrams(passage,unigram_dict_counts)
bigrams_bkwd,bigrams_dict_bkwd,bigrams_dict_probab_bkwd = create_bigrams(passage,unigram_dict_counts,backward=True)
trigrams, trigrams_dict, trigrams_dict_probab = create_trigrams(passage,bigrams_dict)

## Calculation for Bidirectional Bigrams

In [19]:
def get_bidirectional_probability(bidirectional_bigrams,unigram_dict_counts):
  bidirectional_dict_count = {}
  bidirectional_dict_probab = {}
  for i in bidirectional_bigrams:
    if(i in bidirectional_dict_count.keys()):
      bidirectional_dict_count[i] = bidirectional_dict_count[i]+1
    else:
      bidirectional_dict_count[i] = 1

  for i in bidirectional_dict_count.keys():
    try:
      bidirectional_dict_probab[i] = bidirectional_dict_count[i]/get_unigram_count(i[0],unigram_dict_counts)
    except:
      print("Following word has 0 probability in unigrams", i[0])
  return bidirectional_dict_count, bidirectional_dict_probab

In [20]:
bidirectional_bigrams = bigrams+bigrams_bkwd

## Creating Bidirectional Bigrams

In [21]:
bidirectional_dict, bidirectional_dict_probab = get_bidirectional_probability(bidirectional_bigrams,unigram_dict_counts)


## Unigrams Sentence Generation

In [22]:
number_of_stanzas = random.randrange(7,10)

In [26]:
sentences = []
for i in range(1,number_of_stanzas):
    for j in range(1,5):
      sentences.append(generate_unigram_sentence(7,unigram_dict_probability,unigram_sentence=[]))
      if(j%4==0):
        print("\n")

آینہ صدائے زخمہ کي بیکراں گھائل رکھتا 
دہناں کنار چاہیئے بھید خرشید لعنت ستم 
مل رُک اگا روئے پہلو آتی محمد 
شادی بزمخیال سخت ترسنے بیانِ گفتگو پسر 


کے اربابِ ظالم چوکھٹ سبو رکھ کبريا 
اسرار تڑپ برسات فرزانہ مرید بازارِ جلوہٴ 
اک عطا ميں دہقاں بتخانہ کامی نرالے 
نکلوں کوچہ یہاں گزند مشکیں بچا گریے 


ہمتِِ ہوے نرالے بد دوا آنکھوں بیٹھنا 
یورپ تنِ سفلہ لیتا منصور عام بساطِ 
ماہِ آبیارِ کہہ جتنے شاہوں راج شمشیرِ 
گل خودی رقیب کرتے لینا خط مقدور 


پسر کبریا شاعر ربطیٴ فروغ فطرتیِ ختم 
نھیں بتاتا عبرت صحرائیوں دور واقف ساتھی 
لعنت جادو فطرتیِ یہود نطق عزم قط 
پروانہ خوف چڑھتے بدگمانیِ دیوارِ گُل لسان 


امام باتیں جفاخو عنواں روانی دانتوں رو 
مردوکوزندہ وَ مُژدہ ہجوم اسکی کروں تکلف 
راہ معمورے رازِ آتے وجودِ شب مانی 
جادہ ماتمیوں وَ دیکھو آزار لعل اِک 


آسمان کرتا باغ صاحب سویداۓ زلفِ آسکھوں 
اطفال نالٴہ پیوند شعلہٴ گھبرائیں پیٹوں قلمِ 
آہو قریہ نےاردو متاع کباب تاختن قطرہ 
سلیقہ آستیں سچ خداوندگار سہی قرآن چنتی 




### Running evaluation on Unigram

In [25]:
for i in sentences:
  print(i,calculate_perplexity_unigram(i,unigram_dict_probability))

['لیلیِ', 'سببِ', 'فروزاں', 'بوسے', 'روشناس', 'برسرِ', 'خاص'] (5.019378243291581e-32, 72.06941690569462, 9.90245646613125e-14)
['نِگاه', 'راہبر', 'بھیس', 'آسا', 'خاکی', 'ملوکانہ', 'اَبرِ'] (1.5058134729874743e-31, 70.9708046170265, 1.1026607025224699e-13)
['زمامِ', 'قریہ', 'گوارا', 'گرمی', 'زار', 'اسے', 'صاف'] (7.227904670339875e-29, 64.79701851312457, 2.0850421419733638e-13)
['نجف', 'سکتی', 'رائیگاں', 'قبول', 'آیا', 'سرگشہٴ', 'وقف'] (9.637206227119836e-30, 66.81192153366683, 1.6827603223027541e-13)
['چہ', 'مزا', 'رند', 'جُھوٹ', 'تنِ', 'مجبور', 'آشیاں'] (1.3552321256887266e-29, 66.47099494669624, 1.7441135020724725e-13)
['ہاتھوں', 'پھر', 'فتنے', 'کبريا', 'پیٹھ', 'طاؤس', 'حوصلہ'] (9.33604353252234e-29, 64.54108513898737, 2.1436118988852386e-13)
['خالی', 'کاوش', 'گنجینۂ', 'کاشانۂ', 'اہل', 'تسکینِ', 'شرم'] (1.3281274831749521e-28, 64.18861256101972, 2.2273787936400504e-13)
['التفات', 'بجھتی', 'مسرت', 'خمارِ', 'منزلوں', 'سی', 'تاراجِ'] (6.023253891949897e-31, 69.58451025590662, 1.26593665

## Bigrams Sentence Generation

In [27]:
sentences = []
for i in range(1,number_of_stanzas):
    for j in range(1,5):
      sentence = get_bigram_word("123",0,10,bigrams_dict_probab,bigram_word=[])
      sentences.append(sentence)
      if(j%4==0):
        print("\n")

بجا نہ بنے بات کہ یہ قدسیوں سے 
راہ پر ہے نہ ہوتا تو دے رہے 
لرزیدہ عزلت شہہ کے ساتھ عداوت ہی کیوں 
گئے وہ اب وہ بھی نہیں رہا ہے 


پہچانتا پھونکا نہیں رہا حرم ہے نہ ہو 
دی ہے یہ کام بند آیا عجب آزاد 
سنکر ایسی پرواز جہاں میں بھی نہیں رہا 
اپنے خنجر سے گزر بادہ و دل کا 


شہیدِ وفا کر نہیں آتی رہی پیرہن آرائی ضبط اعلان 
سیہ گری میں نے نہ بنے عمر بھر 
پھردھڑکنےکاحوصلہ گاہ خیال بیاباں ماندگی سے ہے نہ نموئے نفس 
محمد وصالِ یار ہوتا گر نظر کا دل 


مژدۂ رگِ جاں ستاں ناوک خیز ہے نہ حاصل سوز 
طَعمہ جذبۂ سمجھ سکتا نہیں ہے کہ یوں 
شرح کہتے ھو تشنگی لبریز نور تھا میں 
ڈال دو عالم گیر ہے یا کہ یوں 


درپردہ میری نفس جادہٴ سر راہ عمل کا 
فضائے زمانہ تنگ شاید کثرتِ نظاّرہ سے دل 
درویش ہوں اور ہے کہ یہ قدسیوں کے 
ہنس لوگ وہی دل کو بھی ہیں ہم 


سامنا چاہتا ہے نہ پوچھ اقبال میں نے 
دولت و نظر ہائے تیز دستی قاتل کیا 
جگنو ملت میں تو نہ ہوتا تو کیا 
اور شے ہےعلم ہے یا کوئی پوچھے ہے 




### Backward Bigrams Sentence Generation

In [28]:
for i in range(1,number_of_stanzas):
    for j in range(1,5):
      sentence = get_bigram_word("123",0,10,bigrams_dict_probab_bkwd)
      if(j%4==0):
        print("\n")

ایسے کب ملاقات بہرِ جز قلندر ڈالے کہہ 
ہائے نوا خونیں دیدۂ میں پردے سوز نیم بالا لانا 
ہائے نوا خونیں دیدۂ میں گھر کے عشق 
معلوم نہیں ایسے کب ملاقات ہے نہیں دل 


گزری سو تھا خدا والو رہنے کے جس 
آسکا دی نے تو ہو کہتے دل سے بھید خاشاک 
ساتھ خاک کا کس تو ہے میں بزم 
پیوند دستاویز کهولنا بیتابی نالہ دیکھی اثر ہے 


ایسی فائدہ سے مجھ نہیں ایسے کب حسن 
نزول ہو بھی کو اس کہ ہے کیا 
قریب بام نظارۂ لطفِ اگر دل وہ کہ 
یار تیغ واں گل برگ تو ہو کہتے 


دریغ لطفِ اگر بھی اور ہوں کرتا کی 
آخر بعد عرصے جتنے جان خوار غم آدمی گلا نمایاں 
عشق غمِ کاوشِ تاراجِ ڈھونڈھنا پار کے خدا 
اچھے کتنے تھے نہ ہے کیا میں دل 


آئنہ ترا یہ کا اس کہ تھا نہ 
شرمندہ میں دل کہ ہیں میں بزم تو 
ميں فکر کہ ہے کیا نے ہم گے شباب آشیانہ 
نفس یاں سے غم کوئی ہر کا اس 


خاموش آتشِ ہے میں گھر اپنے ہے میں 
گوہر قیمتِ میں بزم کہاں ملیں ہم تھے 
سنائیو خاموشی کردار انوکھا کا جس ہے کیا 
تن خوں بہ ہے میں گھر نہ جو 




8

## Bidirectional Bigrams Sentence Generation

In [ ]:
for i in range(1,number_of_stanzas):
    for j in range(1,5):
      get_bigram_word("123",0,10,bidirectional_dict_probab)
      if(j%4==0):
        print("\n")

چراغ محفل ہے محرم ہے صنم کدہ صنم 
گدازِ واں ہو تو ہے وہ عشق کی 
پہنچوں لگا کہ وہ غزنوی وہ عشق کی 
سُراغِ دود چراغ دود تھا میں تڑپ سجدے فضائیں 


صرفہ مالی قفس میں نہ مری کوشش کی 
مقصود یہ کہا تو کہاں سے جس کا 
تکلف ہمیں اٹھائے نہ مری فریاد سے نہیں 
آباد قدر آیا ہے نگاہ کی خامشی میں 


دیمک لیتا کر نہ رہ سکتا سمجھ بلا 
لگاؤ مہجوری کاشانہ رگ خوں جو قمری سے 
جنہں گزرا آئی ہے نگاہ بھی تھا پردہ 
نگہہ ہوائے سیم شوخ نے یہ فضائیں برہنہ 


آزردگاں آزادی ڈھانپا دیار چھوٹوں آتشِ ہے زلف 
دام و شب میں آپ اپنی موج دیکھ 
بڑھتا گھر میں تڑپ رہے گی افلاک مرا 
مشعل ہے 

## Trigrams Sentence Generation

In [60]:
for i in range(1,number_of_stanzas):
    for j in range(1,5):
      get_trigram_word("123","123",0,6,trigrams_dict_probab=trigrams_dict_probab)
      if(j%4==0):
        print("\n")

نہ رکھ امیدِ آزادی دل ز جوئے خوں ہوا کو 
فصل گل مسجد و بھَوں پاس دیکھو تو خانہ ویراں اب وہ 
دم لیا تیرے خوابوں کیا جان برہنہ پائی اے کاش میں 
کچھ اور شے ہےعلم ہے کچھ اور 


پھر صبا وگرنہ شہر ہر گوشۂ اے طفلِ یہ غزل کیا پوجتا 
تاراجِ کاوشِ قلندر جز کھویا گیا حاضر ہوں طاعت میں تھا زندگی 
ہمیشہ تازہ آئیں جس شب فراق ہے یہ قدسیوں سے 
نہ ہوئی عرضِ سر ہے اب مرا انتظار کر 


تیغ ہلال یہ رشک کیا کم ہے 
مشکلیں مجھ نہ تم اب میں اشک پیدا میں پھر گریہ 
مانندِ کہکشاں عدو کے راہِ صحرائے دیکھ کر شرمائں یہود 
صد شکر اعلان جنوں وائے محرومیِ نگاہ پاک جو نہیں کوئی کہے 


یہ شیشہ وہ خاک کہ اندازِ بے طلب غالبؔ خستہ فلک نے 
فرق ان اسد مت درپردہ انھیں جاں دادہٴ ظلمت ھے میرا قد 
یہ لاش غنچۂ ناشگفتہ علم میں دست ساقی مصر و فلسطیں 
عاشقی صبر کسی راستے پرافشاں ہے ورنہ کوئی یہ خلش یار آشنا 


غیر یوں مری نوا دل گزر خطا کس ہمیشہ تازہ شب کو 
ہے کہاں وہ دن کہ وجد ذوق میں 
بس پہلے وگرنہ آگ اب منزلِ سیر کے نگینِ نامِ غم جہاں 
شہادت تھی یہاں ساقی جاتا وگرنہ دل نازک بو بھی ہوتی 


بُھولے تو ہے غنیمت بہ ہر ورنہ کیا دیکھیں ک

# Conclusion and Observations:
* The unigram model was very basic and was not accurate
* The bigram model was better than unigram but not good
* Surprisingly, the backward bigram model also provided a better representation of sentences
* The Bi-directional bigrams performed really well as compared to backward or plain bigram models
* The trigram models were also good but the bidirectoinal bigrams were a clear win.
